<a href="https://colab.research.google.com/github/aruntakhur/Resume/blob/main/Exp_KDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from keras.datasets import mnist
import numpy as np

In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [5]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [29]:
#batch_size = len(train_dataset)
#n_iters = 3000
#num_epochs = n_iters / len(train_dataset) 
#num_epochs = int(num_epochs)
#num_epochs = 3000
#___
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [41]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        print("a")
        self.fc1 = nn.Linear(input_dim, hidden_dim1) 

        # Non-linearity
        self.reluu = nn.ReLU()

        # Linear function
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2) 

        # Non-linearity
        #self.sigmoid = nn.ReLU()

        # Linear function
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3) 

        # Non-linearity
        #self.sigmoid = nn.ReLU()
        
        # Linear function
        self.fc4 = nn.Linear(hidden_dim3, hidden_dim4) 

        # Non-linearity
        #self.sigmoid = nn.ReLU()
        
        # Linear function
        self.fc5 = nn.Linear(hidden_dim4, hidden_dim5) 

        # Non-linearity
        #self.sigmoid = nn.ReLU()

        # Linear function (readout)
        self.fc6 = nn.Linear(hidden_dim5, output_dim)  
        self.softmax = nn.Softmax()
    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.reluu(out)
        
        # Linear function  # LINEAR
        out = self.fc2(x)

        # Non-linearity  # NON-LINEAR
        out = self.reluu(out)
                
        # Linear function  # LINEAR
        out = self.fc3(x)

        # Non-linearity  # NON-LINEAR
        out = self.reluu(out)
                
        # Linear function  # LINEAR
        out = self.fc4(x)

        # Non-linearity  # NON-LINEAR
        out = self.reluu(out)
                
        # Linear function  # LINEAR
        out = self.fc5(x)

        # Non-linearity  # NON-LINEAR
        out = self.reluu(out)

        # Linear function (readout)  # LINEAR
        out = self.fc6(out)
        return out

In [42]:
input_dim = 28*28
#hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, 500, 300, 250, 100, 50, output_dim)

a


In [43]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [44]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 2 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities
                images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

RuntimeError: ignored